# 20.Tensorboard 可视化好帮手

In [1]:
import tensorflow as tf
import numpy as np

## 层定义

In [2]:
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    ## 添加层并返回该层输出
    layer_name = 'layer%s' %n_layer
    with tf.name_scope('layer'):
        
        with tf.name_scope('weights'):  # 权重
            Weights= tf.Variable(tf.random_normal([in_size, out_size]),name='W')
            tf.summary.histogram(layer_name + '/weights', Weights)
        
        with tf.name_scope('biases'):  # 偏置
            biases = tf.Variable(tf.zeros([1,out_size])+0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)
        
        with tf.name_scope('Wx_plus_b'):  # 公式
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)  # y = w * x + b
            if activation_function is None:
                outputs=Wx_plus_b
            else:
                outputs= activation_function(Wx_plus_b)

        tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

## 测试数据

In [3]:
x_data= np.linspace(-1, 1, 300, dtype=np.float32)[:,np.newaxis]  # 最后增加一个维度
noise=  np.random.normal(0, 0.05, x_data.shape).astype(np.float32)  # 噪声，干扰数据
y_data= np.square(x_data) -0.5+ noise  # y = x^2 - 0.5 

## 输入层

In [4]:
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

## 中间层

In [5]:
l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)

## 输出层

In [6]:
prediction= add_layer(l1, 10, 1, n_layer=2, activation_function=None)

## 损失函数

In [7]:
with tf.name_scope('loss'):
    loss= tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))
    
    tf.summary.scalar('loss', loss)  # 添加一loss图

## 训练

In [8]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

## 会话

In [10]:
with tf.Session() as sess:

    init = tf.global_variables_initializer()
    sess.run(init)

    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs/", sess.graph)
    
    for step in range(1000):
        sess.run(train_step, feed_dict={xs: x_data, ys: y_data})  # 执行训练
        
        if step % 50 == 0:  # 每50步执行一次
            result = sess.run(merged, feed_dict={xs: x_data, ys: y_data})
            writer.add_summary(result, step)

## 终端调用

In [ ]:
!tensorboard --logdir logs

TensorBoard 0.1.6 at http://ubuntu-SSD64G:6006 (Press CTRL+C to quit)
W0928 16:17:13.668216 Reloader tf_logging.py:86] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.


## [程序源码网址](https://github.com/MorvanZhou/tutorials/blob/master/tensorflowTUT/tf15_tensorboard/full_code.py)